클래스 가중치만 둠.

In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from sklearn.preprocessing import LabelEncoder
import sklearn
import numpy as np

from tensorflow.keras.callbacks import TensorBoard

tb_callback = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)


2023-08-10 01:46:17.990291: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 01:46:18.049684: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 01:46:18.051231: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 01:46:18.995217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
train_file = '/home/azureuser/projects/aerius/ai/myapp/chatbot/datas/intent5.csv'
data = pd.read_csv(train_file, delimiter = ',')
queries = data['query'].tolist()
intents = data['intent'].tolist()

print(data.head(5))
print('len of queries = ', len(queries))
print('len of intents = ', len(intents))

                              query  intent
0  아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?       0
1             실수로 취소하면 재주문해야 하는 거죠?       0
2                     택배비 따로 추가되나요?       1
3                          택배비 있나요?       1
4                        택배비 따로 들어요       1
len of queries =  235867
len of intents =  235867


In [6]:
encoder = LabelEncoder()
encoder.fit(intents)

encoded_intents = encoder.transform(intents)
num_classes = len(encoder.classes_)
print(f"The number of unique labels is {num_classes}")

The number of unique labels is 4


In [7]:
from ai.myapp.chatbot.utils.Preprocess import Preprocess
p = Preprocess()

words = []
for sentence in queries:
    if isinstance(sentence, str):  # Only process if the sentence is a string
        preprocessed = p.delete_intent_trash_tags(sentence=sentence)
        word_list, _ = p.divide_words_tags(preprocessed)
        words.extend(word_list)
    else:
        print(f"Found non-string value: {sentence}")


# Initialize the tokenizer
p.initialize_tokenizer(words)

# Convert the queries into sequences
sequences = []
for sentence in queries:
    sequence = p.text_to_sequence(sentence)
    sequences.append(sequence)

None


In [8]:
sequence_lengths = [len(seq) for seq in sequences]

# 평균과 표준편차 계산
average_length = np.mean(sequence_lengths)
std_length = np.std(sequence_lengths)

# 평균 + 1x or 2x 표준편차
max_seq_length_1std = int(average_length + std_length)
max_seq_length_2std = int(average_length + 2*std_length)

# 문장 길이의 분포 확인
percentile_95 = np.percentile(sequence_lengths, 95)

print(f"Average Length: {average_length}")
print(f"Std Length: {std_length}")
print(f"Average + 1x Std: {max_seq_length_1std}")
print(f"Average + 2x Std: {max_seq_length_2std}")
print(f"95th Percentile Length: {percentile_95}")

Average Length: 2.434460946211212
Std Length: 1.9924301606156376
Average + 1x Std: 4
Average + 2x Std: 6
95th Percentile Length: 6.0


In [9]:
# # SMOTE 오버샘플링 추가 시작
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(padded_seqs, encoded_intents)

# ds_resampled = tf.data.Dataset.from_tensor_slices((X_resampled, y_resampled))
# ds_resampled = ds_resampled.shuffle(len(X_resampled))

# train_size_resampled = int(len(X_resampled)*0.7)
# val_size_resampled = int(len(X_resampled)*0.2)
# test_size_resampled = int(len(X_resampled)*0.1)

# train_ds_resampled = ds_resampled.take(train_size_resampled).batch(20)
# val_ds_resampled = ds_resampled.skip(train_size_resampled).take(val_size_resampled).batch(20)
# test_ds_resampled = ds_resampled.skip(train_size_resampled + val_size_resampled).take(test_size_resampled).batch(20)

# model.fit(train_ds_resampled, validation_data=val_ds_resampled, epochs=EPOCH, verbose=1, callbacks=[tb_callback], class_weight=class_weights_dict)
# # SMOTE 오버샘플링 추가 종료

In [12]:
from ai.backend.settings import INTENT_MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=INTENT_MAX_SEQ_LEN, padding='post')

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, encoded_intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.tokenizer.word_index) + 1

input_layer = Input(shape=(INTENT_MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=INTENT_MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters = 128,
               kernel_size = 3,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(filters = 128,
               kernel_size = 4,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(filters = 128,
               kernel_size = 5,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(num_classes, name='logits')(dropout_hidden)
predictions = Dense(num_classes, activation=tf.nn.softmax)(logits)

model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])


from sklearn.utils import class_weight

class_weights = class_weight.compute_sample_weight(class_weight='balanced', y=encoded_intents)
class_weights_dict = {i: weight for i, weight in enumerate(np.unique(class_weights))}

model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1, callbacks=[tb_callback], class_weight=class_weights_dict)



Epoch 1/5
8256/8256 [==============================] - 213s 25ms/step - loss: 2.2797 - accuracy: 0.7773 - val_loss: 0.6114 - val_accuracy: 0.7926
Epoch 2/5
8256/8256 [==============================] - 245s 30ms/step - loss: 2.0794 - accuracy: 0.7873 - val_loss: 0.5743 - val_accuracy: 0.8071
Epoch 3/5
8256/8256 [==============================] - 269s 33ms/step - loss: 2.0103 - accuracy: 0.7936 - val_loss: 0.6453 - val_accuracy: 0.8199
Epoch 4/5
8256/8256 [==============================] - 311s 38ms/step - loss: 1.9684 - accuracy: 0.7953 - val_loss: 0.5695 - val_accuracy: 0.8158
Epoch 5/5
8256/8256 [==============================] - 317s 38ms/step - loss: 1.9282 - accuracy: 0.8015 - val_loss: 0.5630 - val_accuracy: 0.8214


In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. 모델 예측
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# test_ds_resampled에서 라벨만 추출
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# 2. 성능 지표 계산
precision = precision_score(y_true, y_pred_classes, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_true, y_pred_classes)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

1180/1180 [==============================] - 11s 9ms/step
Precision: 0.7591
Recall: 0.7034
F1 Score: 0.7167
Accuracy: 0.7034


In [1]:
loss, accuracy = model.evaluate(test_ds, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

NameError: name 'model' is not defined

In [2]:
loss, accuracy = model.evaluate(test_ds, verbose=1)
print(f'Accuracy: {accuracy * 100}')
print(f'loss: {loss}')

model.save('/home/azureuser/projects/aerius/ai/myapp/chatbot/models/intent_model_03_after_class_weight.h5')

NameError: name 'model' is not defined